In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [3]:
img_rows, img_cols = 28, 28
num_classes = 10

In [4]:
def data_prep(raw):
    out_y = keras.utils.np_utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

In [5]:
x,y = data_prep(train_data)

In [6]:
# from sklearn.model_selection import train_test_split

In [7]:
# x, x_val, y, y_val = train_test_split(x, y, test_size = 0.1, random_state=2)

In [8]:
# test data preparation
X_test = test_data.values.astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
X_test = X_test/255

In [9]:
# data augmanetation
data_generator = ImageDataGenerator(rescale = 1/255, rotation_range = 10, 
                                     zoom_range =0.1, width_shift_range = 0.1, 
                                    height_shift_range = 0.1)
data_generator.fit(x)

In [10]:
# CNN Model Building
model = Sequential()

model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [11]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [12]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [13]:
history = model.fit(x,y, batch_size = 128, epochs = 10)

Train on 42000 samples
Epoch 1/10
42000/42000 [==============================] - 632s 15ms/sample - loss: 0.3082 - accuracy: 0.9069
Epoch 2/10
42000/42000 [==============================] - 639s 15ms/sample - loss: 0.0649 - accuracy: 0.9812
Epoch 3/10
42000/42000 [==============================] - 638s 15ms/sample - loss: 0.0450 - accuracy: 0.9866
Epoch 4/10
42000/42000 [==============================] - 637s 15ms/sample - loss: 0.0381 - accuracy: 0.9888
Epoch 5/10
42000/42000 [==============================] - 637s 15ms/sample - loss: 0.0324 - accuracy: 0.9905
Epoch 6/10
42000/42000 [==============================] - 637s 15ms/sample - loss: 0.0294 - accuracy: 0.9921
Epoch 7/10
42000/42000 [==============================] - 638s 15ms/sample - loss: 0.0256 - accuracy: 0.9923
Epoch 8/10
42000/42000 [==============================] - 636s 15ms/sample - loss: 0.0262 - accuracy: 0.9926
Epoch 9/10
42000/42000 [==============================] - 635s 15ms/sample - loss: 0.0232 - accuracy: 0.9

In [14]:
pred = model.predict_classes(X_test, verbose=0)

In [15]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

In [16]:
write_preds(pred, "digit_recognition_submission_14042020_v2.csv")